In [1]:
import pandas as pd
import torchtext
from torchtext.vocab import Vectors
from torchtext.legacy import data
import torch
from torch import nn
import torch.nn.functional as F
from d2l import torch as d2l
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn import metrics
import sys
sys.path.append("D:/Experiment")
from tqdm import tqdm
from MyKu import training
from MyKu import processing
from torchtext.vocab import Vectors
from spacy.lang.en import English

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)


cuda:0


In [2]:
processing.create_OLID()
# processing.set_olid_train_data(processing.ORIGIN_DATASET_PATH + '/OLID')
# processing.set_olid_testA_data(processing.ORIGIN_DATASET_PATH + '/OLID')


In [3]:
# train_data = processing.get_OLID_train_data(processing.OLID_DATASET + '/train.tsv')
# test_data = processing.get_OLID_testA_data(
#     processing.OLID_DATASET + '/testA.tsv', processing.OLID_DATASET + '/labels-levela.csv')


In [4]:
spacy_en = English()

def tokenizer(text):  # create a tokenizer function
    """
    定义分词操作
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

def DataLoader():
    def tokenize(x): return x.split()

    TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True, include_lengths=True, fix_length=40)
    LABEL = data.Field(sequential=False, use_vocab=False)
    # 假如train.csv文件并不是只有两列，比如1、3列是review和polarity，2列是我们不需要的数据，
    # 那么就要添加一个全是None的元组， fields列表存储的Field的顺序必须和csv文件中每一列的顺序对应，

    train_fields = [(None, None), ('tweet', TEXT), ('subtask_a', LABEL)]
    # train_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    train_data= data.TabularDataset(
    path='D:/Experiment/datasets/OLID/train.tsv',
    # path='D:/Experiment/datasets/EXIST2021/train.tsv',
    format='tsv',
    fields=train_fields,
    skip_header=True  # 是否跳过文件的第一行
    )
    test_fields = [(None, None), ('tweet', TEXT), ('label', LABEL)]
    # test_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    test_data= data.TabularDataset(
    path='D:/Experiment/datasets/OLID/testA.tsv',
    # path='D:/Experiment/datasets/EXIST2021/test.tsv',
    format='tsv',
        fields=test_fields,
    skip_header=True  # 是否跳过文件的第一行
    )
    return train_data, test_data, TEXT, LABEL


In [5]:
train_data, test_data, TEXT, LABEL = DataLoader()
# x1, x2 = 1, 1
# for index in train_data:
#     index.task1 = 1 if index.task1 == 'sexist' else 0

# for index in test_data:
#     index.task1 = 1 if index.task1 == 'sexist' else 0

# temp_examples = []
# for index in train_data.examples:
#     if len(index.text) == 0:
#         continue
#     if x1 > 3437:
#         break
#     else:
#         x1 += 1
#     temp_examples.append(index)
# train_data.examples = temp_examples
# temp_examples = []
# for index in test_data.examples:
#     if len(index.text) == 0:
#         continue
#     if x2 > 1000:
#         break
#     else:
#         x2 += 1
#     temp_examples.append(index)
# test_data.examples = temp_examples

vectors = Vectors(name='glove.6B.300d.txt', cache=processing.EMBEDDING_PATH)

TEXT.build_vocab(train_data,  # 建词表是用训练集建，不要用验证集和测试集
                  max_size=400000, # 单词表容量
                  vectors=vectors, # 还有'glove.840B.300d'已经很多可以选
                  unk_init=torch.Tensor.normal_ # 初始化train_data中不存在预训练词向量词表中的单词
)

In [6]:
from operator import truediv

# BiLSTM + Attention 
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, n_class, bidirectional, dropout, **kwargs):
        super(BiRNN, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.encoder = nn.LSTM(
            embed_size, num_hiddens, num_layers=num_layers, bidirectional=True, dropout=dropout, batch_first=True)
        self.n_class = n_class  # 类别数
        self.bidirectional = bidirectional  # 控制是否双向LSTM
        if self.bidirectional:
            self.decoder1 = nn.Linear(num_hiddens * 2, num_hiddens)
            self.decoder2 = nn.Linear(num_hiddens, n_class)
        else:
            self.decoder1 = nn.Linear(num_hiddens * 2, num_hiddens)
            self.decoder2 = nn.Linear(num_hiddens, n_class)
        self.weight_W = nn.Parameter(torch.Tensor(2 * num_hiddens, 2 * num_hiddens))
        self.weight_proj = nn.Parameter(torch.Tensor(2 * num_hiddens, 1))
        self.t = nn.Parameter(torch.Tensor(num_hiddens, 2))
        nn.init.uniform_(self.weight_W, -0.1, 0.1)
        nn.init.uniform_(self.weight_proj, -0.1, 0.1)


    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        # print(embeddings.shape)
        self.encoder.flatten_parameters()
        outputs, _ = self.encoder(embeddings.permute(1, 0, 2))
        # print(outputs.shape)
        u = torch.tanh(torch.matmul(outputs, self.weight_W))
        # print(u.shape)
        att = torch.matmul(u, self.weight_proj)
        att_score = F.softmax(att, dim=1)
        # print(att_score.shape)
        score_x = outputs * att_score
        encoding = torch.sum(score_x, dim=1)
        # encoding = torch.cat((outputs[0], outputs[-1]), dim=1)
        outs = self.decoder1(encoding)
        outs = self.decoder2(outs)
        # print(outs.shape)
        return outs


In [7]:
tt = torch.zeros([2, 10])
ss = torch.ones(10)
s = ss.view(1, 10)
res = tt + s
res

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [8]:
# X = torch.ones((4, 6), dtype=torch.long).to(device=DEVICE)
model = BiRNN(len(TEXT.vocab), 300, 200, 1, 2, True, 0.5)
model.to(DEVICE)

d:\Software\anaconda3\envs\pytorch_env\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


BiRNN(
  (embedding): Embedding(20148, 300)
  (encoder): LSTM(300, 200, batch_first=True, dropout=0.5, bidirectional=True)
  (decoder1): Linear(in_features=400, out_features=200, bias=True)
  (decoder2): Linear(in_features=200, out_features=2, bias=True)
)

In [9]:
pretrained_embeddings = TEXT.vocab.vectors
# print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(300)
model.embedding.weight.data[PAD_IDX] = torch.zeros(300)

In [10]:
train_iter, test_iter = data.BucketIterator.splits(
    (train_data, test_data), batch_size=64, sort_within_batch=True, sort_key=lambda x : len(x.tweet), device=DEVICE
)

In [11]:
def train(model, train_iter, optimizer, loss, epoch):
    model.train()
    epoch_loss = 0
    num_sample = 0
    correct = 0
    for batch in tqdm(train_iter, desc=f"Training Epoch {epoch}", colour='red'):
        optimizer.zero_grad()
        text, text_len = batch.tweet
        label = batch.subtask_a
        # text, text_len = batch.text
        # label = batch.task1
        output = model(text)
        pred_y = torch.argmax(output, dim=1)
        correct += torch.sum(pred_y == label)
        l = loss(output, label)
        l.backward()
        epoch_loss += l.item()
        num_sample += len(batch)
        optimizer.step()
    print(
        f'\tTrain Loss: {epoch_loss / num_sample:.3f} | Train Acc: {correct.float() / num_sample* 100:.2f}%')


def test(model, test_iter):
    true_y, pred_y = [], []
    for batch in tqdm(test_iter, desc=f"Testing", colour='green'):
        text, text_len = batch.tweet
        label = batch.label
        # text, text_len = batch.text
        # label = batch.task1
        with torch.no_grad():
            output = model(text)
            pred_y.extend(output.argmax(dim=1).tolist())
            true_y.extend(label.tolist())
    print(metrics.confusion_matrix(true_y, pred_y))
    print(metrics.classification_report(true_y, pred_y))
    print(f'Acc : {metrics.accuracy_score(true_y, pred_y)}\t F1: {metrics.f1_score(true_y, pred_y, average="macro")}')


In [12]:
lr, num_epochs = 0.0001, 20
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
for epoch in range(1, num_epochs+1):
    train(model, train_iter, optimizer, loss, epoch)
    test(model, test_iter)

Training Epoch 1: 100%|██████████| 207/207 [00:03<00:00, 60.22it/s]


	Train Loss: 0.009 | Train Acc: 68.80%


Testing: 100%|██████████| 14/14 [00:00<00:00, 191.61it/s]


[[590  30]
 [134 106]]
              precision    recall  f1-score   support

           0       0.81      0.95      0.88       620
           1       0.78      0.44      0.56       240

    accuracy                           0.81       860
   macro avg       0.80      0.70      0.72       860
weighted avg       0.81      0.81      0.79       860

Acc : 0.8093023255813954	 F1: 0.7209029888551166


Training Epoch 2: 100%|██████████| 207/207 [00:03<00:00, 66.00it/s]


...

[[573  47]
 [122 118]]
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       620
           1       0.72      0.49      0.58       240

    accuracy                           0.80       860
   macro avg       0.77      0.71      0.73       860
weighted avg       0.79      0.80      0.79       860

Acc : 0.8034883720930233	 F1: 0.7270994695582782


Training Epoch 20: 100%|██████████| 207/207 [00:03<00:00, 67.53it/s]


	Train Loss: 0.001 | Train Acc: 98.67%


Testing: 100%|██████████| 14/14 [00:00<00:00, 215.19it/s]

[[540  80]
 [112 128]]
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       620
           1       0.62      0.53      0.57       240

    accuracy                           0.78       860
   macro avg       0.72      0.70      0.71       860
weighted avg       0.77      0.78      0.77       860

Acc : 0.7767441860465116	 F1: 0.7102425876010782
